## Feed forward basic

Basic equation with just one layer:

$$NN(X) = f( W\cdot X + b)$$
with $f(\cdot)$ a ReLU function:
$$
f(x) = \begin{cases}
    0, & \text{if } x < 0\\
    x, & \text{if } x \geq 0
\end{cases}
$$
and $W$ is a matrix of weights per neuron in the layer with dimension $(|N|_{n+1} \times |N|_n)$
Derivative given the entry:
$$
\frac{dNN_j(X)}{dX_i} = \frac{dNN_j}{df}\frac{df}{dX_i} = 
\begin{cases}
    0, & \text{if } x < 0\\
    W_{ji}, & \text{if } x \geq 0
\end{cases}
$$

Example:

* Given a neural network with 1 layer and 2 neurons in the layer we have:
$$
W = \begin{bmatrix}
    1 & 2 & 3\\
    3 & 4 & 6
\end{bmatrix} 
$$

$$
b = \begin{bmatrix}
    1 \\
    3 
\end{bmatrix} 
$$

$$
X = \begin{bmatrix}
    1 \\
    2 \\
    3
\end{bmatrix} 
$$

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-03-07 19:15:49.853242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
def relu(X):
    """_summary_

    Args:
        X (_type_): _description_

    Returns:
        _type_: _description_
    """
    return tf.keras.activations.relu(X).numpy()

def relu_derivative(X, partial_x = 0, partial_j = 0):
    """_summary_

    Args:
        X (_type_): _description_

    Returns:
        _type_: _description_
    """
    return X[partial_j][partial_x]

In [4]:
input_layer = keras.Input(shape = (4,), name='input_nn')
dense_unit = layers.Dense(
    units = 2,
    activation = 'relu',
    name = 'dense_layer'
)(input_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[dense_unit],
    name = 'test_model'
)

In [32]:
# Obtener los pesos de la capa oculta
w = custom_model.get_layer('dense_layer').get_weights()[0].T
b = custom_model.get_layer('dense_layer').get_weights()[1].T

In [68]:
x = tf.convert_to_tensor(
    np.array(range(-13,-1)).reshape((4,3)),
    dtype = tf.float32    
)

In [69]:
hand_result = tf.transpose(relu(tf.matmul(w, x)))
model_output = custom_model(tf.transpose(x))

In [70]:
hand_result, model_output

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 2.114374 , 12.523226 ],
        [ 1.0487409, 11.261716 ],
        [ 0.       , 10.000206 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 2.114374 , 12.523226 ],
        [ 1.0487409, 11.261716 ],
        [ 0.       , 10.000206 ]], dtype=float32)>)

In [71]:
# Test 
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(tf.transpose(xs))
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})

In [72]:
grads['x']

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.25807047, -0.25807047, -0.8092563 ],
       [-0.13669562, -0.13669562, -0.00675702],
       [-0.72874   , -0.72874   , -0.05111217],
       [-1.2036369 , -1.2036369 , -0.39438462]], dtype=float32)>

In [73]:
jacobian['x']

<tf.Tensor: shape=(3, 2, 4, 3), dtype=float32, numpy=
array([[[[ 0.55118585,  0.        ,  0.        ],
         [-0.1299386 ,  0.        ,  0.        ],
         [-0.6776278 ,  0.        ,  0.        ],
         [-0.80925226,  0.        ,  0.        ]],

        [[-0.8092563 ,  0.        ,  0.        ],
         [-0.00675702,  0.        ,  0.        ],
         [-0.05111217,  0.        ,  0.        ],
         [-0.39438462,  0.        ,  0.        ]]],


       [[[ 0.        ,  0.55118585,  0.        ],
         [ 0.        , -0.1299386 ,  0.        ],
         [ 0.        , -0.6776278 ,  0.        ],
         [ 0.        , -0.80925226,  0.        ]],

        [[ 0.        , -0.8092563 ,  0.        ],
         [ 0.        , -0.00675702,  0.        ],
         [ 0.        , -0.05111217,  0.        ],
         [ 0.        , -0.39438462,  0.        ]]],


       [[[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        

In [66]:
custom_grads = np.zeros((
    2,4
))
for j in range(2):
    for i in range(4):
        custom_grads[j,i] = relu_derivative(w, partial_x = i, partial_j = j)

In [67]:
custom_grads

array([[ 0.55118585, -0.1299386 , -0.6776278 , -0.80925226],
       [-0.80925632, -0.00675702, -0.05111217, -0.39438462]])